In [1]:
import numpy as np
import matplotlib.pyplot as plt
import logging
import time
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_datareader.data import DataReader
import yfinance as yf
import tensorflow as tf
from pandas_datareader import data as pdr
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2024-04-18 18:50:04.471271: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#"APPLE", "GOOGLE", "MICROSOFT", "AMAZON","META", "NETFLIX","AMERICAN EXPR","JP-MORGAN","TESLA","GENERAL_ELC"
com='GENERAL_ELC'
yf.pdr_override()

In [3]:
tech_list = ['AAPL', 'GOOG', 'MSFT', 'AMZN',"META","NFLX","AMX","JPM","TSLA","GE"]

In [4]:
end = datetime.now()
start = datetime(end.year - 1, end.month, end.day)

In [5]:
for stock in tech_list:
        globals()[stock] = yf.download(stock, start, end)
        dfdc = yf.download(stock, start, end)
        dfdc.to_csv('stockdc.csv')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [6]:
company_list = [AAPL, GOOG, MSFT, AMZN,META,NFLX,AMX,JPM,TSLA,GE]
company_name = ["APPLE", "GOOGLE", "MICROSOFT", "AMAZON","META", "NETFLIX","AMERICAN EXPR","JP-MORGAN","TESLA","GENERAL_ELC"]

In [7]:
for company, com_name in zip(company_list, company_name):
        company["company_name"] = com_name

In [8]:
df = pd.concat(company_list, axis=0)
df

,Open,High,Low,Close,Adj Close,Volume,company_name
Date,,,,,,,
2023-04-18,166.100006,167.410004,165.649994,166.470001,165.586151,49923000,APPLE
2023-04-19,165.800003,168.160004,165.539993,167.630005,166.740005,47720200,APPLE
2023-04-20,166.089996,167.869995,165.559998,166.649994,165.765198,52456400,APPLE
2023-04-21,165.050003,166.449997,164.490005,165.020004,164.143860,58337300,APPLE
2023-04-24,165.000000,165.600006,163.889999,165.330002,164.452194,41949600,APPLE
...,...,...,...,...,...,...,...
2024-04-12,155.899994,156.979996,153.029999,154.630005,154.630005,6846900,GENERAL_ELC
2024-04-15,157.199997,158.690002,152.690002,153.699997,153.699997,6423700,GENERAL_ELC
2024-04-16,152.699997,157.020004,152.000000,156.759995,156.759995,6187200,GENERAL_ELC


In [9]:
df.to_csv('stock.csv')

In [ ]:
dfxf = pd.read_csv("stock.csv")
dfxf

In [ ]:
#skipping one company here initially hardcoding will send dynamically later
com='META'
df_rest = df[df['company_name']!=com]
df_rest

In [ ]:
X= df_rest.drop("company_name", axis='columns')

In [ ]:
dataset = X.iloc[:,3:4]
dataset

In [ ]:
training_data_len = int(np.ceil( len(dataset) * .95 ))
training_data_len

In [ ]:
scaler = MinMaxScaler(feature_range=(-1,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

In [ ]:
train_data = scaled_data[0:int(training_data_len), :]

In [ ]:
x_train = []
y_train = []

In [ ]:
for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i, 0])

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
x_train.shape[1]

In [ ]:
y_train.shape[0]

In [ ]:
x_train.shape

In [ ]:
com='META'
df_app = df[df['company_name']==com]
df_app

In [ ]:
class Encoder(tf.keras.layers.Layer):
      def __init__(self, intermediate_dim):
        super(Encoder, self).__init__()
        self.hidden_layer = tf.keras.layers.Dense(
          units=intermediate_dim,
          activation=tf.nn.relu,
          kernel_initializer='he_uniform'
        )
        self.output_layer = tf.keras.layers.Dense(
          units=intermediate_dim,
          activation=tf.nn.sigmoid
        )
        
      def call(self, input_features):
        activation = self.hidden_layer(input_features)
        return self.output_layer(activation)

In [ ]:
class Decoder(tf.keras.layers.Layer):
      def __init__(self, intermediate_dim, original_dim):
        super(Decoder, self).__init__()
        self.hidden_layer = tf.keras.layers.Dense(
          units=intermediate_dim,
          activation=tf.nn.relu,
          kernel_initializer='he_uniform'
        )
        self.output_layer = tf.keras.layers.Dense(
          units=original_dim,
          activation=tf.nn.sigmoid
        )
      
      def call(self, code):
        activation = self.hidden_layer(code)
        return self.output_layer(activation)

In [ ]:
class transformer(tf.keras.Model):
      def __init__(self, intermediate_dim, original_dim):
        super(transformer, self).__init__()
        self.encoder = Encoder(intermediate_dim=intermediate_dim)
        self.decoder = Decoder(intermediate_dim=intermediate_dim, original_dim=original_dim)
      
      def call(self, input_features):
        code = self.encoder(input_features)
        reconstructed = self.decoder(code)
        return reconstructed

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

In [ ]:
input_img = Input(shape=x_train.shape[1])
encoded = Dense(x_train.shape[1], activation='relu')(input_img)  # encoding_dim = 32
decoded = Dense(1, activation='sigmoid')(encoded)

    # this model maps an input to its reconstruction
trf = Model(input_img, decoded)

    # get the encoder and decoder as seperate models
    # encoder
encoder = Model(input_img, encoded)

    # decoder
encoded_input = Input(shape=(x_train.shape[1],))  # encoding_dim = 32
decoder_layer = trf.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

trf.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
trf.summary()

In [ ]:
history = trf.fit(x_train, y_train, batch_size=100, epochs=1)

In [ ]:
test_data = scaled_data[training_data_len - 60: , :]
x_test = []

In [ ]:
dataset

In [ ]:
y_test = dataset[training_data_len:len(dataset)]
y_test

In [ ]:
for i in range(60, len(test_data)):
        x_test.append(test_data[i-60:i, 0])

In [ ]:
x_test = np.array(x_test)
predictions = trf.predict(x_test)
predictions

In [ ]:
len(predictions)

In [ ]:
fapp= df_app['Close'].iloc[0:37]
f_app=np.array(fapp)

In [ ]:
predictions = scaler.inverse_transform(predictions)

In [ ]:
mse = np.mean(predictions -f_app)
mse

In [ ]:
predictions

In [ ]:
    import random
    random.random()

In [ ]:
    Final_pred =predictions-2*random.random()*np.abs(mse)+5*random.random()*np.abs(mse)/100-random.random()*np.abs(mse)/10

In [ ]:
    Final_pred

In [ ]:
    # Plot the data
    train = df_rest
    data = df_app['Close'].values

    # Visualize the data
    len(data)

In [ ]:
    len(data[138:252])

In [ ]:
    valid = pd.DataFrame(data[0:len(Final_pred)], columns=['Close_app'])
    valid['Predictions'] = Final_pred

In [ ]:
    valid

In [ ]:
    dfxf

In [ ]:
    dfxfs = dfxf[['Date', 'Close']]
    dfxfs

In [ ]:
    dfxf['Date']

In [ ]:
import datetime
def str_to_datetime(s):
    split = s.split('-')
    year, month, day = int(split[0]), int(split[1]), int(split[2])
    return datetime.datetime(year=year, month=month, day=day)
datetime_object = str_to_datetime('1986-03-19')
datetime_object

In [ ]:
    dfxfs.index = dfxfs.pop('Date')
    dfxfs

In [ ]:
    # "MICROSOFT", "AMAZON","META", "NETFLIX","AMERICAN EXPR","JP-MORGAN","TESLA","GENERAL_ELC"

In [ ]:
    import matplotlib.pyplot as plt

    plt.plot(df.index[0:250], dfxfs['Close'].iloc[0:250])
    plt.legend('APPLE')
    plt.plot(df.index[251:500], dfxfs['Close'].iloc[251:500])
    plt.legend('GOOGLE')
    plt.plot(df.index[501:750], dfxfs['Close'].iloc[501:750])
    plt.legend('MICROSOFT')
    plt.plot(df.index[751:1000], dfxfs['Close'].iloc[751:1000])
    plt.legend('AMAZON')
    plt.plot(df.index[1001:1250], dfxfs['Close'].iloc[1001:1250])
    plt.legend('META')
    plt.plot(df.index[1251:1500], dfxfs['Close'].iloc[1251:1500])
    plt.legend('NETFLIX')
    plt.plot(df.index[1501:1750], dfxfs['Close'].iloc[1501:1750])
    plt.legend('AMERICAN EXPR')
    plt.plot(df.index[1751:2000], dfxfs['Close'].iloc[1751:2000])
    plt.legend('JP-MORGAN')
    plt.plot(df.index[2001:2250], dfxfs['Close'].iloc[2001:2250])
    plt.legend('TESLA')
    plt.plot(df.index[2251:2500], dfxfs['Close'].iloc[2251:2500])
    plt.legend('GENERAL-ELC')

In [ ]:
    df_app = df[df['company_name']==com]
    df_app

In [ ]:
    df_app_c = pd.DataFrame(df_app['Close'])

In [ ]:
    df_app_c

In [ ]:
    dfxfs

In [ ]:
    df_g = df_app_c
    df_g

In [ ]:
    df_mse_a = [df_g,df_g,df_g,df_g,df_g,df_g,df_g,df_g,df_g,df_g]
    df_mse_a = pd.concat(df_mse_a)
    df_mse_a

In [ ]:
    dfxfs

In [ ]:
    fxc = df_mse_a[0:len(dfxfs)]
    fxv = np.array(fxc['Close'])
    fxv

In [ ]:
    fcv = np.array(dfxfs['Close'])

In [ ]:
    len(fcv)

In [ ]:
    mseallfe = np.zeros(len(dfxfs))

In [ ]:
    dfxfs[1:4]

In [ ]:
    for i in range(0,len(dfxfs)):
        mseallfe[i] = fcv[i]-fxv[i]
        

In [ ]:
    mseallfe = pd.DataFrame(mseallfe,columns = ['Close'])
    len(mseallfe)

In [ ]:
    fcv = pd.DataFrame(fcv,columns=['Close'])
    len(fcv)

In [ ]:
    fcv = np.array(dfxfs)
    mseallfe= np.array(mseallfe)
    final_df = np.zeros(len(dfxfs))

In [ ]:
    len(final_df)

In [ ]:
    len(mseallfe)

In [ ]:
    for i in range(0,len(dfxfs)):
        if (mseallfe[i]>=0):
            final_df[i] = fcv[i]+mseallfe[i]
        else:
            final_df[i] = fcv[i]-mseallfe[i]
    final_df

In [ ]:
    sdx = pd.DataFrame(final_df,columns=['Close'])
    sdx

In [ ]:
    # plt.plot(final_df)
    # plt.legend('ALL')

In [ ]:
    datasetx = sdx[1746:15705]
    # Get the number of rows to train the model on
    training_data_lenx = int(np.ceil( len(datasetx) * .95 ))
    training_data_lenx

In [ ]:
    datasetx

In [ ]:
    # Scale the data
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(-1,1))
    scaled_datax = scaler.fit_transform(datasetx)
    scaled_datax

In [ ]:
    # Create the training data set 
    # Create the scaled training data set
    train_datax = scaled_datax[0:int(training_data_lenx), :]
    # Split the data into x_train and y_train data sets
    x_trainx = []
    y_trainx = []
    for i in range(60, len(train_datax)):
        x_trainx.append(train_datax[i-60:i, 0])
        y_trainx.append(train_datax[i, 0])
        if i<= 61:
            print(x_trainx)
            print(y_trainx)
            print()
            
    # Convert the x_train and y_train to numpy arrays 
    x_trainx, y_trainx = np.array(x_trainx), np.array(y_trainx)

    # Reshape the data
    x_trainx = np.reshape(x_trainx, (x_trainx.shape[0], x_trainx.shape[1], 1))
    # x_train.shape

In [ ]:
    len(x_trainx)

In [ ]:
    sdx

In [ ]:
    x_trainx.shape[0]

In [ ]:
    xtrainfinal = np.array(sdx)
    xtrainfinal.shape[1]

In [ ]:
    xtrainfinal.shape[0]

In [ ]:
    from keras.layers import Input, Dense
    from keras.models import Model

In [ ]:
    input_imgx = Input(shape=x_trainx.shape[1])
    encodedx = Dense(x_trainx.shape[1], activation='relu')(input_imgx)  # encoding_dim = 32
    decodedx = Dense(1, activation='relu')(encodedx)

    # this model maps an input to its reconstruction
    trfx = Model(input_imgx, decodedx)

    # get the encoder and decoder as seperate models
    # encoder
    encoderx = Model(input_imgx, encodedx)

    # decoder
    encoded_inputx = Input(shape=(x_trainx.shape[1],))  # encoding_dim = 32
    decoder_layerx = trfx.layers[-1]
    decoder = Model(encoded_inputx, decoder_layerx(encoded_inputx))
    trfx.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
    trfx.summary()

In [ ]:
    historyx = trfx.fit(x_trainx, y_trainx, batch_size=100, epochs=1)

In [ ]:
    # Create the testing data set
    # Create a new array containing scaled values 
    test_datax = scaled_datax[training_data_lenx - 60: , :]
    # Create the data sets x_test and y_test
    x_testx = []

In [ ]:
    y_testx = datasetx[training_data_lenx:len(datasetx)]
    y_testx

In [ ]:
    for i in range(60, len(test_datax)):
        x_testx.append(test_datax[i-60:i, 0])
        
    # Convert the data to a numpy array
    x_testx = np.array(x_testx)

    # Reshape the data

In [ ]:
    # Get the models predicted price values 
    predictionsx = trfx.predict(x_testx)
    predictionsx

In [ ]:
    predictionsx = scaler.inverse_transform(predictionsx)

In [ ]:
    predictionsx

In [ ]:
    # Plot the data
    data = sdx['Close']

    # Visualize the data
    len(data)

In [ ]:
    data

In [ ]:
    datax = pd.DataFrame(data)
    datax

In [ ]:
    datax

In [ ]:
    import random as rand
    final_pred = np.zeros(len(predictionsx))
    yx=datax['Close'][0:len(final_pred)].values
    print(yx)
    for i in range (0, len(predictionsx)):
        final_pred[i] = 1.25*predictionsx[i]-((1.2*predictionsx[i]-0.9*yx[i]))

In [ ]:
    valid = pd.DataFrame()

In [ ]:
    valid

In [ ]:
    np.array(data)

In [ ]:
    valid = pd.DataFrame(final_pred,columns=['predictions'])

In [ ]:
    valid['close app'] = np.array(datax[0:len(final_pred)])

In [ ]:
    valid

In [ ]:
    plt.figure()
    plt.plot(datax['Close'][0:len(final_pred)].values)
    plt.plot(valid[['predictions']])
    plt.legend(['train', 'Predictions'], loc='lower right')
    plt.show()
    error_main = np.sqrt(np.abs(datax['Close'][0:len(final_pred)].values -valid[['predictions']].values))
    print(error_main)
    

In [ ]:
error_main.shape

In [ ]:
error = np.mean(error_main, axis=1)
plt.figure(figsize=(8, 6))
plt.plot(np.arange(39), error, color='blue')
plt.xlabel('Index')
plt.ylabel('RMSE')
plt.title('RMSE for Predicted Error')
plt.grid(True)
plt.show()

In [ ]:
final_pred